# Create Images of Sprites

Load in the sprite data

In [3]:
import re
sprites_files = ["../iridisalpha/src/graphics/sprites.asm",
                 "../iridisalpha/src/graphics/bonusphase_sprites.asm",
                 "../iridisalpha/src/graphics/enemy_sprites.asm"]
sprites_data = {}
for sprites_file in sprites_files:
    input_file = open(sprites_file,'r')
    sprite_data = []
    for l in input_file.readlines():
        if "SPRITE" in l:
            if sprite_data:
                sprites_data[sprite_name] = sprite_data
            sprite_name = l[18:20].strip().lower()
            sprite_data = []
            continue

        m = re.findall(r"[0-1]{24}",l)
        if not m:
            continue
        bits = m[0]
        sprite_line = []
        for i in range(0,23,2):
            bitpair = bits[i:i+2]
            sprite_line += [bitpair]
            sprite_line += [bitpair]
        sprite_data += [sprite_line]
    if sprite_data:
        sprites_data[sprite_name] = sprite_data


Get the color data from the sprite movements file

In [27]:
import gzip
log_file = "IridisAlphaTitleEnemySpriteMovements.txt.gz"
input_file = gzip.open(log_file,'rt')

movements = [ [] for _ in range(8) ]
for l in input_file.readlines():
    # the x and y co-ords for each sprite
    if "C:d000" in l:
        s = l[9:59]
        s = s.replace(' ',',').replace(',,',',')
        s = s.split(',')
        co_ords = [(int(s[i],16), int(s[i+1],16)) for i in range(0,len(s),2)]
        continue

    # sprite_colors
    if "C:d025" in l:
        s = l[9:40]
        s = s.replace(' ',',').replace(',,',',')
        sprite_colors_array = s.split(',')
        multi_color0 = sprite_colors_array[0]
        multi_color1 = sprite_colors_array[1]
        sprite_colors = sprite_colors_array[2:]
        continue

    # the most significant bit for each sprite's x co-ord
    if "C:d010" in l:
        msb = int(l[9:11],16)
        continue

    # The current sprite value
    if "C:07f8" in l:
        s = l[9:33]
        s = s.replace(' ',',').replace(',,',',')
        sprite_values = s.split(',')
        continue
        
    # current planet
    # change back to 78b0 in unmodified rom
    if "C:78b3" in l:
        planet = int(l[9:11],16)
        continue

    # current level
    # 49c6 in unmodified rom
    if not "C:49c9" in l:
        continue
    s = l[9:33]
    s = s.replace(' ',',').replace(',,',',')
    levels = s.split(',')
    level = int(levels[planet],16)

    # Add the most significant bit to each sprite's x value
    for i,v in enumerate([1,2,4,8,16,32,64,128]):
        if msb & v:
            x,y = co_ords[i]
            co_ords[i] = (x+256,y)

    for i, spr in enumerate(sprite_values):
        movements[i] += [(sprite_values[i], co_ords[i], (planet,level),
                         (multi_color0,multi_color1,sprite_colors[i]))]


Function for actually drawing the sprite

In [34]:
from PIL import Image, ImageColor

def paintSprite(origin, pixels, sprite, colors):
    multicol0, multicol1,color = colors
    colormap = {
        "01": multicol0,
        "10": color,
        "11": multicol1,
    }
    
    x_o, y_o = origin
    if sprite not in sprites_data:
        print(sprite)
        return
    bit_array = sprites_data[sprite]
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "00":
                continue
            pixel_color = ImageColor.getrgb(fhex_to_rgb[colormap[bit]])
            pixels[x_o+x, y_o+y] = pixel_color



Write out the sprites to image files

In [49]:
import os
from collections import defaultdict
from sprite_map import sprite_map 
from colors_map import *

bits=8
width = 24
height = 21

for j, mvt in enumerate(movements):
    for i,(sprite, _origin, (_planet,_level), colors) in enumerate(mvt):
        if sprite not in sprite_map:
            continue
        sprite_name = sprite_map[sprite]
        fname = f'sprites/sprite_{sprite_name}_{"".join(colors)}.png'
        if os.path.isfile(fname):
            continue
        img = Image.new( 'RGBA', (width,height))
        pixels = img.load()
        origin = (0,0)
        paintSprite(origin, pixels, sprite, colors)
        img = img.resize((img.width * 10, img.height * 10), Image.NEAREST)
        img.save(fname)

